# Artificial Neural Networks and Deep Learning

---

## Homework 1: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 1` in your Google Drive.
2. Upload the `training_set.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 1 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.zip` file to Codabench.


## ⚙️ Import Libraries

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split
import random
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

## ⏳ Load and inspect the data

In [ ]:
data = np.load('training_set.npz')
X = data['images']
y = data['labels']

# Check class imbalance
unique_labels, counts = np.unique(y, return_counts=True)
print("Unique labels:", unique_labels)
print("Counts:", counts)

X = (X).astype('float32')
# Convert to one hoot encoding
y = tfk.utils.to_categorical(y)

print('Before data points filter shape:', X.shape, y.shape)

import json
with open('training-data-filter/blacklist.json', 'r') as file:
	blacklist = json.load(file)
blacklist = sorted(blacklist['blacklist'])
X = np.delete(X, blacklist, axis=0)
y = np.delete(y, blacklist, axis=0)

print('After data points filter shape:', X.shape, y.shape)

train_size = int(X.shape[0] * 0.9)
val_size = int(X.shape[0] * 0.0)
test_size = X.shape[0] - train_size - val_size

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=y)

print(X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape)

In [ ]:
# Labels
labels = {
	0: "Basophil",
	1: "Eosinophil",
	2: "Erythroblast",
	3: "Immature granulocytes",
	4: "Lymphocyte",
	5: "Monocyte",
	6: "Neutrophil",
	7: "Platelet"
}

In [ ]:
# Function to apply random brightness using tf.image
def random_brightness(image, max_delta=0.25):
    # Adjust brightness by a random factor in the range [-max_delta, max_delta]
    return tf.image.adjust_brightness(image, max_delta)


plt.imshow(random_brightness(X_train_val[0] / 255))

In [ ]:
# Inspect data
# Display a sample of images from the training-validation dataset
num_img = 10
random_indices = random.sample(range(len(X_train_val)), num_img)

fig, axes = plt.subplots(1, num_img, figsize=(20, 20))

def get_label(y):
    index = np.where(y == 1)[0]
    return labels[int(index)]

# Iterate through the selected number of images
for i, idx in enumerate(random_indices):
    ax = axes[i % num_img]
    ax.imshow(np.squeeze(X_train_val[idx] / 255), vmin=0., vmax=1.)
    ax.set_title(get_label(y_train_val[idx]))
    ax.axis('off')

# Adjust layout and display the images
plt.tight_layout()
plt.show()

## 🛠️ Train and Save the Model

In [ ]:
def build_model():
    efficientNetV2 = tfk.applications.EfficientNetV2L(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=X_train_val[0].shape,
        classes=len(labels),
        pooling='avg',
        include_preprocessing=True,
        #name="efficientnetv2-l",
    )
    efficientNetV2.trainable = False
    # Define input layer with shape matching the input images
    inputs = tfk.Input(shape=X_train_val[0].shape, name='input_layer')

    # Apply data augmentation for training robustness
    augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(0.25),                   # Rotation up to ±25%
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.RandomZoom(0.1),                        # Zoom in or out up to 10%
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),  # Horizontal and vertical flip
        tf.keras.layers.RandomBrightness(factor=0.2),           # Brightness adjustment up to ±20%
    ], name='preprocessing')

    x = augmentation(inputs)

    # Pass augmented inputs through the MobileNetV3Small feature extractor
    x = efficientNetV2(x)

    # Add a dropout layer for regularisation
    x = tfkl.Dropout(0.3, name='dropout')(x)

    # Add final Dense layer for classification with softmax activation
    outputs = tfkl.Dense(y_train_val.shape[-1], activation='softmax', name='dense')(x)

    # Define the complete model linking input and output
    tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Compile the model with categorical cross-entropy loss and Adam optimiser
    tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=1e-4), metrics=['accuracy'])

    return tl_model

# Display a summary of the model architecture
#tl_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(tl_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Perform K CV

from sklearn.model_selection import KFold

N = 2

# Initialize lists to store training histories, scores, and best epochs
histories = []
scores = []
best_epochs = []
patience = 20

# Create a KFold cross-validation object
kfold = KFold(n_splits=N, shuffle=True, random_state=seed)

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(X_train_val, y_train_val)):
    print("Starting training on fold num: {}".format(fold_idx+1))

    # Build a new dropout model for each fold
    k_model = build_model()

    # Create an EarlyStopping callback
    early_stopping = tfk.callbacks.EarlyStopping(
        monitor='val_accuracy', mode='max', patience=patience, restore_best_weights=True
    )

    callbacks = [early_stopping]

    # Train the model on the training data for this fold
    history = k_model.fit(
    x = X_train_val[train_idx],
    y = y_train_val[train_idx],
    validation_data=(X_train_val[valid_idx], y_train_val[valid_idx]),
    batch_size = 1024,
    epochs = 400,
    callbacks = callbacks,
    verbose = 1
    ).history

    # Evaluate the model on the validation data for this fold
    score = k_model.evaluate(X_train_val[valid_idx], y_train_val[valid_idx], verbose=1)
    scores.append(score[1])

    # Calculate the best epoch for early stopping
    best_epoch = len(history['loss']) - patience
    best_epochs.append(best_epoch)

    # Store the training history for this fold
    histories.append(history)

In [ ]:
# Define a list of colors for plotting
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# Print mean and standard deviation of MSE scores
print("Accuracy")
print(f"Mean: {np.mean(scores).round(4)}\nStd:  {np.std(scores).round(4)}")

# Create a figure for MSE visualization
plt.figure(figsize=(15,6))

# Plot MSE for each fold
for fold_idx in range(N):
    plt.plot(histories[fold_idx]['val_accuracy'][:-patience], color=colors[fold_idx], label=f'Fold N°{fold_idx+1}')
    plt.title('Mean Squared Error')
    plt.legend(loc='upper right')
    plt.grid(alpha=.3)

# Show the plot
plt.show()

In [ ]:
# Calculate the average best epoch
avg_epochs = int(np.mean(best_epochs))
print(f"Best average epoch: {avg_epochs}")

In [ ]:
# Build the final model using the calculated average best epoch
final_model = build_model()

# Train the final model on the combined training and validation data
final_history = final_model.fit(
    x = X_train_val,
    y = y_train_val,
    batch_size = 2048,
    epochs = avg_epochs
).history

In [ ]:
# Evaluate and plot the performance of the final model on the test data
print('Final Model Test Performance')
loss, acc = final_model.evaluate(X_test, y_test, verbose=2)
print(loss, acc)

# Save the model
from datetime import datetime
model_filename = f'efficientNetV2L[test{acc}][{datetime.now().strftime("%y%m%d_%H%M%S")}].keras'

final_model.save(model_filename)

## 👔 Load a trained model (if needed!)

In [ ]:
tl_model = tf.keras.models.load_model('KaggleEfficientNetV2L85.1241109_182031.keras')

## ✍🏿 Make evaluation

In [ ]:
#loss, acc = tl_model.evaluate(X_test, y_test, verbose=2)
#print('Model, accuracy: {:5.2f}%'.format(100 * acc))

# Predict labels for the entire test set
predictions = tl_model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Convert predictions to class labels
pred_classes = np.argmax(predictions, axis=-1)

# Extract ground truth classes
true_classes = np.argmax(y_test, axis=-1)

# Calculate and display test set accuracy
accuracy = accuracy_score(true_classes, pred_classes)
print(f'Accuracy score over the test set: {round(accuracy, 4)}')

# Calculate and display test set precision
precision = precision_score(true_classes, pred_classes, average='weighted')
print(f'Precision score over the test set: {round(precision, 4)}')

# Calculate and display test set recall
recall = recall_score(true_classes, pred_classes, average='weighted')
print(f'Recall score over the test set: {round(recall, 4)}')

# Calculate and display test set F1 score
f1 = f1_score(true_classes, pred_classes, average='weighted')
print(f'F1 score over the test set: {round(f1, 4)}')

# Compute the confusion matrix
cm = confusion_matrix(true_classes, pred_classes)

# Combine numbers and percentages into a single string for annotation
annot = np.array([f"{num}" for num in cm.flatten()]).reshape(cm.shape)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm.T, annot=annot, fmt='', xticklabels=list(labels.values()), yticklabels=list(labels.values()), cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('KaggleEfficientNetV2L90.46.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py KaggleEfficientNetV2L90.46.keras